<img style="float: right;" src="images/hyperstream.svg">

# HyperStream Tutorial 5: Workflows

Workflows define a graph of streams. Usually, the first stream will be a special "raw" stream that pulls in data from a custom data source. Workflows can have multiple time ranges, which will cause the streams to be computed on all of the ranges given.

## Introduction

In this tutorial, we will be ussing a time-series dataset about the temperature in different countries and cities. The dataset is availabel at [The Census at School New Zeland][1]. The necessary files for this tutorial are already included in the folder **data/TimeSeriesDatasets_130207**.

In particular, there are four files with the minimum and maximum temperatures in different cities of Asia, Australia, NZ and USA from 2000 to 2012. And the rainfall levels of New Zeland. 

![workflows](images/workflow_world_temp.svg)

[1]: http://new.censusatschool.org.nz/resource/time-series-data-sets-2013/

In [1]:
try:
    %load_ext watermark
    watermark = True
except ImportError:
    watermark = False
    pass

import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream
from hyperstream import TimeInterval
from hyperstream.utils import UTC
import hyperstream

from datetime import datetime
from utils import plot_high_chart
from utils import plot_multiple_stock
from dateutil.parser import parse

if watermark:
    %watermark -v -m -p hyperstream -g

hs = HyperStream(loglevel=30)
print(hs)
print([p.channel_id_prefix for p in hs.config.plugins])

2017-07-28 14:36:05,636 [WARNI]  Plate C is empty during the creation
2017-07-28 14:36:05,637 [WARNI]  Plate C.C is empty during the creation


CPython 2.7.6
IPython 5.3.0

hyperstream 0.3.6

compiler   : GCC 4.8.4
system     : Linux
release    : 3.19.0-80-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : 36d4562f14e67feeb92988c1803154979715e928
HyperStream version 0.3.6, connected to mongodb://localhost:27017/hyperstream, session id <no session>
[u'example', u'data_importers', u'data_generators']


## Reading the data

In the data folder there are four csv files with the names **TempAsia.csv, TempAustralia.csv, TempNZ.csv and TempUSA.csv**. The first column of each csv file contains a header with the names of the columns. The first one being the date and the following are the minimum and maximum temperature in different cities with the format **cityMin** and **cityMax**.

Here is an example of the first 5 rows of the **TempAsia.csv** file:

```
Date,TokyoMax,TokyoMin,BangkokMax,BangkokMin
2000M01,11.2,4.2,32.8,24
```

The format of the date has the form **YYYYMmm** where **YYYY** is the year and **mm** is the month. Because this format is not recognized by the default parser of the **csv_reader** tool, we will need to specify our own parser that first replaces the **M** by an hyphen **-** and then applies the **dateutils.parser**.

Then, we will use a tool to read each csv, and a Stream to store all the results of applying the tool. When we specify to the tool that there is a header row in the csv file, the value of each Stream instance will be a dictionary with the name of the column and its corresponding value. For example, a Stream instance with the 4 cities shown above will look like:

```
[2000-01-19 00:00:00+00:00]: {'BangkokMin': 24.0, 'BangkokMax': 32.8, 'TokyoMin': 4.2}
```

In [2]:
def dateparser(dt):
    return parse(dt.replace('M', '-')).replace(tzinfo=UTC)

ti_all = TimeInterval(datetime(1999, 1, 1).replace(tzinfo=UTC),
                      datetime(2013, 1, 1).replace(tzinfo=UTC))
ti_sample = TimeInterval(datetime(2007, 1, 1).replace(tzinfo=UTC),
                         datetime(2007, 3, 1).replace(tzinfo=UTC))

# M will be the Memory Channel
M = hs.channel_manager.memory

countries_list = ['Asia', 'Australia', 'NZ', 'USA']

## Create the plates and meta_data instances

In [3]:
countries_dict = {
    'Asia': ['BangkokMin', 'BangkokMax', 'HongKongMax', 'HongKongMin', 'KualaLumpurMax', 'KualaLumpurMin',
             'NewDelhiMax', 'NewDelhiMin', 'TokyoMax', 'TokyoMin'],
    'Australia': ['BrisbaneMax', 'BrisbaneMin', 'Canberramax', 'CanberraMin', 'GoldCoastMax', 'GoldCoastMin',
                  'MelbourneMin', 'Melbournemax',  'SydneyMax', 'SydneyMin'],
    'NZ': ['AucklandMax', 'AucklandMin', 'ChristchurchMax', 'ChristchurchMin', 'DunedinMax', 'DunedinMin',
           'HamiltonMax', 'HamiltonMin','WellingtonMax', 'WellingtonMin'],
    'USA': ['ChicagoMin', 'ChicagoMax', 'HoustonMax', 'HoustonMin', 'LosAngelesMax', 'LosAngelesMin',
            'NYMax', 'NYMin', 'SeattleMax', 'SeattleMin']
}

# delete_plate requires the deletion to be first childs and then parents
for plate_id in ['C.C', 'C']:
    if plate_id in [plate[0] for plate in hs.plate_manager.plates.items()]:
        hs.plate_manager.delete_plate(plate_id=plate_id, delete_meta_data=True)

for country in countries_dict:
    id_country = 'country_' + country
    if not hs.plate_manager.meta_data_manager.contains(identifier=id_country):
        hs.plate_manager.meta_data_manager.insert(
            parent='root', data=country, tag='country', identifier=id_country)
    for city in countries_dict[country]:
        id_city = id_country + '.' + 'city_' + city
        if not hs.plate_manager.meta_data_manager.contains(identifier=id_city):
            hs.plate_manager.meta_data_manager.insert(
                parent=id_country, data=city, tag='city', identifier=id_city)
            
C = hs.plate_manager.create_plate(plate_id="C", description="Countries", values=[], complement=True,
                                  parent_plate=None, meta_data_id="country")
CC = hs.plate_manager.create_plate(plate_id="C.C", description="Cities", values=[], complement=True,
                                   parent_plate="C", meta_data_id="city")

In [4]:
print hs.plate_manager.meta_data_manager.global_plate_definitions


root[root:None]
╟── country[country_NZ:NZ]
║   ╟── city[country_NZ.city_AucklandMax:AucklandMax]
║   ╟── city[country_NZ.city_AucklandMin:AucklandMin]
║   ╟── city[country_NZ.city_ChristchurchMax:ChristchurchMax]
║   ╟── city[country_NZ.city_ChristchurchMin:ChristchurchMin]
║   ╟── city[country_NZ.city_DunedinMax:DunedinMax]
║   ╟── city[country_NZ.city_DunedinMin:DunedinMin]
║   ╟── city[country_NZ.city_HamiltonMax:HamiltonMax]
║   ╟── city[country_NZ.city_HamiltonMin:HamiltonMin]
║   ╟── city[country_NZ.city_WellingtonMax:WellingtonMax]
║   ╙── city[country_NZ.city_WellingtonMin:WellingtonMin]
╟── country[country_Australia:Australia]
║   ╟── city[country_Australia.city_BrisbaneMax:BrisbaneMax]
║   ╟── city[country_Australia.city_BrisbaneMin:BrisbaneMin]
║   ╟── city[country_Australia.city_Canberramax:Canberramax]
║   ╟── city[country_Australia.city_CanberraMin:CanberraMin]
║   ╟── city[country_Australia.city_GoldCoastMax:GoldCoastMax]
║   ╟── city[country_Australia.city_GoldCoastMin

## Create the workflow and execute it

In [5]:
from hyperstream import Workflow

# parameters for the csv_mutli_reader tool
csv_params = dict(
    filename_template='data/TimeSeriesDatasets_130207/Temp{}.csv',
    datetime_parser=dateparser, skip_rows=0, header=True)

sum_values = lambda x: sum(x.values())

with Workflow(workflow_id='tutorial_05', name='tutorial_05', owner='tutorials',
              description='Tutorial 5 workflow', online=False) as w:

    country_node = w.create_node(stream_name='raw_data', channel=M, plates=[C])
    city_node = w.create_node(stream_name='temperature', channel=M, plates=[CC])
    # FIXME Are these correct?
    country_node_avg_temp = w.create_node(stream_name='avg_temp', channel=M, plates=[C])
    world_node_avg_temp = w.create_node(stream_name='avg_temp', channel=M, plates=[])

    for c in C:
        country_node[c] = hs.plugins.data_importers.factors.csv_multi_reader(source=None, **csv_params)
        for cc in CC[c]:
            city_node[cc] = hs.factors.splitter_from_stream(source=country_node[c],
                                                            splitting_node=country_node[c],
                                                            use_mapping_keys_only=True)
        ## FIXME tool_factory_function() takes 3 arguments... 0 given?
        ##      87         if alignment_node and alignment_node.plates:
        ##     88             # TODO: Need to implement alignment nodes that live inside plates
        ##---> 89             raise NotImplementedError("Currently only alignment nodes outside of plates are supported")
        ##     90 
        ##     91         self.alignment_node = alignment_node
        ## 
        ##NotImplementedError: Currently only alignment nodes outside of plates are supported
        #country_node_avg_temp[c] = hs.factors.aggregate(sources=[city_node], 
        #                                               alignment_node=country_node_avg_temp,
        #                                               aggregation_meta_data='city',
        #                                               func=sum)
        ## FIXME this option does not work either
        #country_node_avg_temp[c] = hs.factors.aggregate_into_dict_and_apply(sources=[city_node], 
        #                                               alignment_node=country_node_avg_temp,
        #                                                aggregation_meta_data='city',
        #                                                func=sum_values)
    world_node_avg_temp = hs.factors.aggregate_into_dict_and_apply(sources=[city_node], 
                                                        alignment_node=country_node_avg_temp,
                                                        aggregation_meta_data='city',
                                                        func=sum_values)
    w.execute(ti_all)

In [6]:
for key, value in world_node_avg_temp.iteritems():
    print("[{}]: {}".format(key, value))

[alignment_node]: Node(node_id='avg_temp', plates=[Plate(meta_data_id=u'country', plate_id=u'C')])
[sources]: [Node(node_id='temperature', plates=[Plate(meta_data_id=u'city', plate_id=u'C.C')])]
[tool]: AggregateIntoDictAndApply(aggregation_meta_data='city', func=<function <lambda> at 0x7f845c51e9b0>)
[workflow]: Workflow(description='Tutorial 5 workflow', factors=[MultiOutputFactor(input_plate=None, output_plates=(Plate(meta_data_id=u'country', plate_id=u'C'),), sink=Node(node_id='raw_data', plates=[Plate(meta_data_id=u'country', plate_id=u'C')]), source=None, splitting_node=None, tool=CsvMultiReader(datetime_column=0, datetime_parser=<function dateparser at 0x7f845c51ed70>, filename_template='data/TimeSeriesDatasets_130207/Temp{}.csv', header=True, skip_rows=0)), MultiOutputFactor(input_plate=Plate(meta_data_id=u'country', plate_id=u'C'), output_plates=(Plate(meta_data_id=u'city', plate_id=u'C.C'),), sink=Node(node_id='temperature', plates=[Plate(meta_data_id=u'city', plate_id=u'C.C'

## See the country raw data streams

In [7]:
# Print the results
for stream_id, stream in country_node.streams.iteritems():
    print(stream_id)
    print(stream.window().first())

(('country', 'NZ'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'WellingtonMin': 14.2, 'ChristchurchMin': 10.8, 'HamiltonMin': 12.4, 'DunedinMax': 18.2, 'WellingtonMax': 20.0, 'ChristchurchMax': 20.2, 'AucklandMax': 23.4, 'HamiltonMax': 23.8, 'DunedinMin': 8.8, 'AucklandMin': 15.5})
(('country', 'USA'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'ChicagoMax': 1.8, 'LosAngelesMin': 10.0, 'HoustonMax': 21.6, 'NYMax': 4.6, 'SeattleMax': 7.9, 'SeattleMin': 1.4, 'ChicagoMin': -8.1, 'NYMin': -5.6, 'HoustonMin': 7.4, 'LosAngelesMax': 19.6})
(('country', 'Australia'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'BrisbaneMax': 28.2, 'MelbourneMin': 15.9, 'Melbournemax': 24.3, 'BrisbaneMin': 19.1, 'CanberraMin': 10.1, 'GoldCoastMax': 27.5, 'SydneyMax': 24.9, 'Canberramax': 24.5, 'GoldCoastMin': 20.2, 'SydneyMin': 17.7})
(('country', 'Asia'),)
StreamInstance(timestamp=dateti

In [8]:
for stream_id, stream in city_node.streams.iteritems():
    print(stream_id)
    print(stream.window().first())

(('country', 'Australia'), ('city', 'GoldCoastMax'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=27.5)
(('country', 'Asia'), ('city', 'TokyoMax'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=11.2)
(('country', 'Asia'), ('city', 'BangkokMin'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=24.0)
(('country', 'Asia'), ('city', 'HongKongMin'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=14.3)
(('country', 'Asia'), ('city', 'BangkokMax'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=32.8)
(('country', 'Asia'), ('city', 'KualaLumpurMin'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=23.5)
(('country', 'NZ'), ('city', 'WellingtonMin'))
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value=14.2)
(('country', 'Australia'), ('city', 'Canberr

In [9]:
this_country = 'Australia'
this_country_cities_temp = M.find_streams(country=this_country, name='temperature')
this_country_cities_names = []
data = []
time = []
for city_id, city_stream in this_country_cities_temp.iteritems():
    meta_data = dict(city_id.meta_data)
    this_country_cities_names.append(meta_data['city'])
    data.append([instance.value for instance in city_stream.window().items()])
    time.append([str(instance.timestamp) for instance in city_stream.window().items()])

plot_multiple_stock(data, time=time, names=this_country_cities_names,
                    title='Temperatures in ' + this_country, ylabel='ºC')

In [10]:
from pprint import pprint
pprint(w.to_dict(tool_long_names=False))

{'factors': [{'id': 'csv_multi_reader', 'sink': 'raw_data', 'sources': []},
             {'id': 'splitter_from_stream',
              'sink': 'temperature',
              'sources': ['raw_data']}],
 'nodes': [{'id': 'raw_data'}, {'id': 'temperature'}, {'id': 'avg_temp'}],
 'plates': {u'C': [{'id': 'raw_data', 'type': 'node'}],
            u'C.C': [{'id': 'temperature', 'type': 'node'}]}}


In [11]:
print(w.to_json(w.factorgraph_viz, tool_long_names=False, indent=4))

{
    "nodes": [
        {
            "type": "rv",
            "id": "raw_data"
        },
        {
            "type": "rv",
            "id": "temperature"
        },
        {
            "type": "rv",
            "id": "avg_temp"
        },
        {
            "type": "fac",
            "id": "csv_multi_reader"
        },
        {
            "type": "fac",
            "id": "splitter_from_stream"
        }
    ],
    "links": [
        {
            "source": "csv_multi_reader",
            "target": "raw_data"
        },
        {
            "source": "raw_data",
            "target": "splitter_from_stream"
        },
        {
            "source": "splitter_from_stream",
            "target": "temperature"
        }
    ]
}


In [12]:
temp_tools_csv = {}
temp_streams = {}
for country in countries_list:
    temp_tools_csv[country] = hs.plugins.example.tools.csv_reader(
            'data/TimeSeriesDatasets_130207/Temp{}.csv'.format(country),
            header=True, dateparser=dateparser)
    temp_streams[country] = M.get_or_create_stream(country)
    temp_tools_csv[country].execute(sources=[], sink=temp_streams[country],
                                    interval=ti_all)

## Print one Stream Instance per Stream

Now that we have generated one Stream per each country, we can inspect the first Stream Instance of each Stream.

In [13]:
for country in countries_list:
    print(temp_streams[country])
    print(temp_streams[country].window().first())

Stream(stream_id=Asia, channel_id=memory)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'NewDelhiMax': 20.1, 'NewDelhiMin': 8.1, 'HongKongMin': 14.3, 'KualaLumpurMin': 23.5, 'TokyoMax': 11.2, 'KualaLumpurMax': 32.2, 'HongKongMax': 19.5, 'BangkokMin': 24.0, 'BangkokMax': 32.8, 'TokyoMin': 4.2})
Stream(stream_id=Australia, channel_id=memory)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'BrisbaneMax': 28.2, 'MelbourneMin': 15.9, 'Melbournemax': 24.3, 'BrisbaneMin': 19.1, 'CanberraMin': 10.1, 'GoldCoastMax': 27.5, 'SydneyMax': 24.9, 'Canberramax': 24.5, 'GoldCoastMin': 20.2, 'SydneyMin': 17.7})
Stream(stream_id=NZ, channel_id=memory)
StreamInstance(timestamp=datetime.datetime(2000, 1, 28, 0, 0, tzinfo=<UTC>), value={'WellingtonMin': 14.2, 'ChristchurchMin': 10.8, 'HamiltonMin': 12.4, 'DunedinMax': 18.2, 'WellingtonMax': 20.0, 'ChristchurchMax': 20.2, 'AucklandMax': 23.4, 'HamiltonMax': 23.8, 'DunedinMin': 8.8, 'Au

## Visualize the temperatures in one Country

Now, we can visualize the temperatures of all the cities in one country. First, we will create a list of all the cities in one of the Streams by looking at the first Stream Instance. Then, we will create a list of lists containing the temperature value of each city, together with their corresponding time. Then, we can use the function **plot_multiple_stock** created for this tutorial.

In [14]:
one_country_name = countries_list[0]

# The following code is only for the visualization
this_cities_names = [key for key, value in temp_streams[one_country_name].window().items()[0].value.iteritems()]

data = {city:[] for city in this_cities_names}
time = []
for key, values in temp_streams[one_country_name].window().items():
    time.append(str(key))
    for city, temperature in values.iteritems():
        data[city].append(temperature)
        
data = [value for key, value in data.iteritems()]
        
plot_multiple_stock(data, time=time, names=this_cities_names,
                    title='Temperatures in ' + one_country_name, ylabel='ºC')

# Create a Stream with all the city names

In order to split the raw streams that contain as a value a dictionary with every city and its corresponding temperature, we need to use a splitting tool. The splitting_from_stream tool is a MultiOutputTool that expects a stream with the keys that are used to split the original stream, and a list of streams that will act as sink.

## Create the mapping names

Here, we will generate first a stream with all the keys that we want to use for the split

In [15]:
one_country_stream = temp_streams[one_country_name]

# It is similar to a database channel
A = hs.channel_manager.assets
this_cities_stream = A.get_or_create_stream('cities_{}'.format(one_country_name))

mapping = {}
for city in this_cities_names:
    mapping[city] = city

print mapping

{'NewDelhiMax': 'NewDelhiMax', 'NewDelhiMin': 'NewDelhiMin', 'HongKongMin': 'HongKongMin', 'KualaLumpurMin': 'KualaLumpurMin', 'TokyoMax': 'TokyoMax', 'KualaLumpurMax': 'KualaLumpurMax', 'HongKongMax': 'HongKongMax', 'BangkokMin': 'BangkokMin', 'BangkokMax': 'BangkokMax', 'TokyoMin': 'TokyoMin'}


## Create the Stream with the names

Now we can write the mapping into the stream that will be used with the splitting tool

In [16]:
from hyperstream import StreamInstance
from hyperstream import StreamId

A.write_to_stream(stream_id=this_cities_stream.stream_id, data=StreamInstance(ti_all.start, mapping))

print this_cities_stream.window(TimeInterval.up_to_now()).items()

2017-07-28 14:36:07,394 [WARNI]  Found duplicate document: Tried to save duplicate unique keys (E11000 duplicate key error collection: hyperstream.streams index: stream_id_1_datetime_1 dup key: { : { name: "cities_Asia", meta_data: [] }, : new Date(915148800000) })
2017-07-28 14:36:07,398 [WARNI]  Stream cities_Asia not available for time interval (0001-01-01 00:00:00+00:00, 2017-07-28 13:36:07.398000+00:00]. Perhaps upstream calculations haven't been performed


[StreamInstance(timestamp=datetime.datetime(1999, 1, 1, 0, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x7f846c5eda90>), value={u'NewDelhiMax': u'NewDelhiMax', u'NewDelhiMin': u'NewDelhiMin', u'HongKongMin': u'HongKongMin', u'KualaLumpurMin': u'KualaLumpurMin', u'TokyoMax': u'TokyoMax', u'KualaLumpurMax': u'KualaLumpurMax', u'HongKongMax': u'HongKongMax', u'BangkokMin': u'BangkokMin', u'BangkokMax': u'BangkokMax', u'TokyoMin': u'TokyoMin'})]


## Creating a city plate

We are creating here one node per city

In [17]:
for city in this_cities_names:
    if not hs.plate_manager.meta_data_manager.contains(identifier='city_'+city):
        print("Adding " + city)
        hs.plate_manager.meta_data_manager.insert(parent='root', data=city,
                                                  tag='city', identifier='city_'+city)

Adding NewDelhiMax
Adding NewDelhiMin
Adding HongKongMin
Adding KualaLumpurMin
Adding TokyoMax
Adding KualaLumpurMax
Adding HongKongMax
Adding BangkokMin
Adding BangkokMax
Adding TokyoMin


## Creating the sink Streams

Now, we will create one Stream per each city. Then, we will use all these Streams as a sink when ussing the splitting tool.

In [18]:
cities_plate = hs.plate_manager.create_plate(plate_id='C', meta_data_id='city', parent_plate=None, 
                                             values=[], complement=True, description='Cities')
this_country_temps = []
for city in this_cities_names:
    print("Adding " + city)
    this_country_temps.append(M.get_or_create_stream(stream_id=StreamId(name='temperature',
                                                                         meta_data=(('city', city),))))

Adding NewDelhiMax
Adding NewDelhiMin
Adding HongKongMin
Adding KualaLumpurMin
Adding TokyoMax
Adding KualaLumpurMax
Adding HongKongMax
Adding BangkokMin
Adding BangkokMax
Adding TokyoMin


It is possible to create all the Streams passing a list to the splitter tool **splitter_from_list**. However, this could not be automated in a workflow

```Python
# TODO Ussing this new tool, it is not necessary to create a new stream. However, if it is a Stream it could be
# automated for any other countries
splitter_tool = hs.plugins.example.tools.splitter_from_list(element=None)

# TODO try to change the parameter name of MultiOutputTool splitting_stream to splitting_parameter
# or something that does not force you to think that it is a stream
splitter_tool.execute(source=one_country_stream, splitting_stream=this_cities_list, output_plate=cities_plate, 
                      interval=ti_all, input_plate_value=None, sinks=this_country_temps)
```

**TODO: Ask Tom: Question:** With the splitter_from_stream version we still need to create a list with the mapping... Then, I can not see the difference between using one or the other method.
**Answer:** The list in the Stream is allowed to change over time, this makes the future Streams to be more robust to change (e.g. the number of houses in the SPHERE project). Also, there is a tool that uses a dictionary for the splitting criteria, that is **splitter_of_dict** that expects the splitter_stream to be None, and a mapping parameter containing the static mapping.

In [19]:
splitter_tool = hs.tools.splitter_from_stream(element=None, use_mapping_keys_only=False)

splitter_tool.execute(source=one_country_stream, splitting_stream=this_cities_stream, output_plate_values=cities_plate, 
                      interval=ti_all, sinks=this_country_temps, meta_data_id="city")

2017-07-28 14:36:07,453 [WARNI]  Stream cities_Asia not available for time interval (0001-01-01 00:00:00+00:00, 2013-01-01 00:00:00+00:00]. Perhaps upstream calculations haven't been performed


In [20]:
one_city = this_country_temps[0]
city_name = one_city.stream_id.meta_data[0][1]

print one_city
print one_city.window(ti_all)

my_time, my_data = zip(*[(key.__str__(), value) for key, value in one_city.window(ti_all).items()])

plot_high_chart(my_time, my_data, type="high_stock", title='Temperature in {}'.format(city_name), yax='ºC')

2017-07-28 14:36:07,561 [WARNI]  Stream temperature: [city=NewDelhiMax] not available for time interval (1999-01-01 00:00:00+00:00, 2013-01-01 00:00:00+00:00]. Perhaps upstream calculations haven't been performed


Stream(stream_id=temperature: [city=NewDelhiMax], channel_id=memory)
StreamView(force_calculation=False, stream=Stream(stream_id=temperature: [city=NewDelhiMax], channel_id=memory), time_interval=TimeInterval(start=datetime.datetime(1999, 1, 1, 0, 0, tzinfo=<UTC>), end=datetime.datetime(2013, 1, 1, 0, 0, tzinfo=<UTC>)))
